In [1]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data

In [2]:
# dataset_cora = Planetoid(root='./cora/', name='Cora')
# # dataset = Planetoid(root='./citeseer',name='Citeseer')
# # dataset = Planetoid(root='./pubmed/',name='Pubmed')
# print(dataset_cora[0])

In [2]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1, 0, 1], [0, 1, 2], [1, 2, 3]], dtype=torch.float)
dataset_cora = Data(x=x, edge_index=edge_index)
dataset_cora

Data(x=[3, 3], edge_index=[2, 4])

In [3]:
class mydata(Data):
    def __init__(self, x, edge_index, edge_attr=None, y=None, pos=None, time=None, **kwargs):
        super(mydata, self).__init__(x, edge_index, y, edge_attr, pos, time, **kwargs)

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1, 0, 1], [0, 1, 2], [1, 2, 3]], dtype=torch.float)
dataset_cora = mydata(x=x, edge_index=edge_index)
dataset_cora.num_nodes

3

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(3, 16, aggr='mean')
        self.conv2 = SAGEConv(16, 2, aggr='mean')

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [6]:
model = Net()
print(model)

Net(
  (conv1): SAGEConv(3, 16, aggr=mean)
  (conv2): SAGEConv(16, 2, aggr=mean)
)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora.to(device)
print(data)

cuda
mydata(x=[3, 3], edge_index=[2, 4])


In [8]:
x, edge_index = data.x, data.edge_index
edge_index.shape

torch.Size([2, 4])

In [9]:
import torch.optim as optim

In [10]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [11]:
model.train()
for epoch in range(10):
    out = model(data)
    # loss = criterion(out[data.train_mask], data.y[data.train_mask])

    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()
    print(epoch)
    # _, pred = torch.max(out[data.train_mask], dim=1)
    # correct = (pred == data.y[data.train_mask]).sum().item()
    # acc = correct/data.train_mask.sum().item()

    # print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
    #     epoch, loss.item(), acc))

0
1
2
3
4
5
6
7
8
9


In [ ]:
# model.eval()
# out = model(data)
# loss = criterion(out[data.test_mask], data.y[data.test_mask])
# _, pred = torch.max(out[data.test_mask], dim=1)
# correct = (pred == data.y[data.test_mask]).sum().item()
# acc = correct/data.test_mask.sum().item()
# print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 1.9127 test_acc: 0.2790
